# Lab 1: Linear Regression and Overfitting

### Machine Learning and Pattern Recognition, September 2015

* The lab exercises should be made in groups of two or three people.
* The deadline is sunday September 20, 23:59.
* Assignment should be sent to Philip Versteeg. (p.j.j.p.versteeg@uva.nl) The subject line of your email should be "\#lab\_lastname1\_lastname2\_lastname3".
* Put your and your teammates' names in the body of the email
* Attach the .IPYNB (IPython Notebook) file containing your code and answers. Naming of the file follows the same rule as the subject line. For example, if the subject line is "lab01\_Kingma\_Hu", the attached file should be "lab01\_Kingma\_Hu.ipynb". Only use underscores ("\_") to connect names, otherwise the files cannot be parsed.
* Make sure we can run your notebook / scripts!

Notes on implementation:

* You should write your code and answers in this IPython Notebook: http://ipython.org/notebook.html. If you have problems, please contact me.
* Please write your answers right below the questions.
* Among the first lines of your notebook should be "%pylab inline". This imports all required modules, and your plots will appear inline.
* Refer to last week's lab notes, i.e. http://docs.scipy.org/doc/, if you are unsure about what function to use. There are different correct ways to implement each problem!
* For this lab, your regression solutions should be in closed form, i.e., should not perform iterative gradient-based optimization but find the exact optimum directly.

$\newcommand{\bPhi}{\mathbf{\Phi}}$
$\newcommand{\bx}{\mathbf{x}}$
$\newcommand{\bw}{\mathbf{w}}$
$\newcommand{\bt}{\mathbf{t}}$
$\newcommand{\by}{\mathbf{y}}$
$\newcommand{\bm}{\mathbf{m}}$
$\newcommand{\bS}{\mathbf{S}}$
$\newcommand{\bI}{\mathbf{I}}$

## Part 1: Polynomial Regression

### 1.1. Generate sinusoidal data (5 points)
Write a method `gen_sinusoidal(N)` that generates toy data like in fig 1.2 of the MLPR book. The method should have a parameter $N$, and should return $N$-dimensional vectors $\bx$ and $\bt$, where $\bx$ contains evenly spaced values from 0 to (including) 2$\pi$, and the elements $t_i$ of $\bt$ are distributed according to:

$$t_i \sim \mathcal{N}(\mu_i, \sigma^2)$$

where $x_i$ is the $i$-th elements of $\bf{x}$, the mean $\mu_i = sin(x_i)$ and the standard deviation $\sigma = 0.2$.


In [ ]:
%pylab inline

In [ ]:
def gen_sinusoidal(N):
    x = linspace(0, 2*pi, N)
    t = random.normal(sin(x), .2)
    return x, t

In [ ]:
x, t = gen_sinusoidal(500)
plot(x, t, '+')

### 1.2 Polynomial regression (15 points)

Write a method `fit_polynomial(x, t, M)` that finds the maximum-likelihood solution of an _unregularized_ $M$-th order polynomial for some dataset `x`. The error function to minimize w.r.t. $\bw$ is:

$E(\bw) = \frac{1}{2} (\bPhi\bw - \bt)^T(\bPhi\bw - \bt)$

where $\bPhi$ is the _feature matrix_ (or _design matrix_) as explained in the MLPR book at section 3.1.1, $\bt$ is the vector of target values. Your method should return a vector $\bw$ with the maximum-likelihood parameter estimates. 

In [ ]:
def fit_polynomial(x, t, M):
    # Alternative: linalg.inv( Phi.T.dot(Phi) ).dot(Phi.T)
    Phi = fliplr(vander(x, M+1)) # M+1 for M'th order polynomial
    return linalg.pinv(Phi).dot(t)  # pinv = Moore-Penrose inverse

### 1.3 Plot (5 points)
Sample a dataset with $N=9$, and fit four polynomials with $M \in (0, 1, 3, 9)$.
For each value of $M$, plot the prediction function, along with the data and the original sine function. The resulting figure should look similar to fig 1.4 of the MLPR book. Note that you can use matplotlib's `plt.pyplot(.)` functionality for creating grids of figures.

In [ ]:
def show_regression(x, t, w, xs=False):
    if xs == False:
        xs = linspace(0, 2*pi, 100)
    plot(xs, polyval(w[::-1], xs), '-r', lw=2)
    plot(x, t, 'o')
    plot(xs, sin(xs), '-g')
    show()

In [ ]:
x, t = gen_sinusoidal(9)
    
for M in [0, 1, 3, 9]:
    w = fit_polynomial(x, t, M)
    show_regression(x, t, w)

### 1.4 Regularized linear regression (10 points)

Write a method `fit_polynomial_reg(x, t, M, lamb)` that fits a _regularized_ $M$-th order polynomial to the sinusoidal data, as discussed in the lectures, where `lamb` is the regularization term _lambda_. (Note that 'lambda' cannot be used as a variable name in Python since it has a special meaning). The error function to minimize w.r.t. $\bw$:

$E(\bw) = \frac{1}{2} (\bPhi\bw - \bt)^T(\bPhi\bw - \bt) + \frac{\lambda}{2} \mathbf{w}^T \mathbf{w}$

For background, see section 3.1.4 of the MLPR book.

In [ ]:
def fit_polynomial_reg(x, t, M, lamb):
    Phi = fliplr( vander(x, M+1) )
    return linalg.inv( lamb * eye(M+1) + Phi.T.dot(Phi) ).dot(Phi.T).dot(t)






### 1.5 Model selection by cross-validation (10 points)
Use cross-validation to find a good choice of $M$ and $\lambda$, given a dataset of $N=9$ datapoints generated with `gen_sinusoidal(9)`. You should write a function that tries (loops over) a reasonable range of choices of $M$ and $\lambda$, and returns the choice with the best cross-validation error. In this case you can use $K=9$ folds, corresponding to _leave-one-out_ crossvalidation.

You can let $M \in (0, 1, ..., 10)$, and let $\lambda \in (e^{-10}, e^{-9}, ..., e^{0})$.

To get you started, here's a method you can use to generate indices of cross-validation folds.

In [ ]:
def kfold_indices(N, k):
    all_indices = np.arange(N,dtype=int)
    np.random.shuffle(all_indices)
    idx = np.floor(np.linspace(0,N,k+1))
    train_folds = []
    valid_folds = []
    for fold in range(k):
        valid_indices = all_indices[idx[fold]:idx[fold+1]]
        valid_folds.append(valid_indices)
        train_folds.append(np.setdiff1d(all_indices, valid_indices))
    return train_folds, valid_folds

In [ ]:
def select_model(x, t):

    N = len(x)
    train_folds, valid_folds = kfold_indices(N, N)
    Ms = range(0,11)
    lambdas = [exp(-10 + i) for i in range(0,11)];
    errors = zeros((len(Ms), len(lambdas)))

    for i, M in enumerate(Ms):
        for j, lamb in enumerate(lambdas):
            err = []
            for k, idx in enumerate(train_folds):

                # Regularized regression on training data
                train_xs = x[idx]
                train_ts = t[idx]
                w = fit_polynomial_reg(train_xs, train_ts, M+1, lamb)

                # Compare regression with validation data using RMS error
                valid_x = x[valid_folds[k]][0]
                valid_t = t[valid_folds[k]][0]
                estimate_t = polyval(w[::-1], valid_x)

                err += [sqrt(mean(square(estimate_t - valid_t)))]
            errors[i,j] = mean(err)

    # Transpose: Ms vertically and lambdas horizontally
    errors = errors.T
    min_id = argmin(errors)
    M_id = min_id % errors.shape[1] # horizontal position
    lamb_id = min_id // errors.shape[1] # vertical position

    # Plot this
    fig, ax = plt.subplots(figsize=(6,5))
    heatmap = ax.pcolor(log(errors), cmap=plt.cm.Blues)

    # Highlight optimal choice
    ax.add_patch(matplotlib.patches.Rectangle(
            (M_id, lamb_id), 1, 1, facecolor="none", edgecolor='red', linewidth=2))

    ax.set_xticks(arange(len(Ms))+0.5, minor=False);
    ax.set_yticks(arange(len(lambdas))+0.5, minor=False);
    ax.set_xticklabels(Ms, minor=False);
    ax.set_yticklabels(['$\exp('+str(int(e))+')$' for e in log(lambdas)], minor=False);
    cbar = plt.colorbar(heatmap)
    title('$\log($error$)$ for different values of $\lambda$ and $M$.\
            \n$\lambda=\exp(%s)$ and $M=%s$ is optimal (red square)\n'
            % (log(lambdas[lamb_id]), Ms[M_id]))
    show()
    
    return Ms[M_id], lambdas[lamb_id]

Create a comprehensible plot of the cross-validation error for each choice of $M$ and $\lambda$. Highlight the best choice. 

_Question_: Explain over-fitting and underfitting, illuminated by your plot. Explain the relationship with model bias and model variance.

Answer:

### 1.6 Plot best cross-validated fit (5 points)

For some dataset with $N = 9$, plot the model with the optimal $M$ and $\lambda$ according to the cross-validation error, using the method you just wrote. Let the plot make clear which $M$ and $\lambda$ were found.

In [ ]:
x,t = gen_sinusoidal(9)
M, lamb =  select_model(x, t)
w = fit_polynomial_reg(x, t, M, lamb)

title('Regularized least squares regression \n\
    using $\lambda=\exp(%s)$ and $M=%s$' % (log(lamb), M))
show_regression(x, t, w)


## Part 2: Bayesian Linear (Polynomial) Regression

### 2.1 Sinusoidal data 2 (5 points)

Write a function `gen_sinusoidal2(N)` that behaves identically to `gen_sinusoidal(N)` except that the generated values $x_i$ are not linearly spaced, but drawn from a uniform distribution between $0$ and $2 \pi$.

In [ ]:
def gen_sinusoidal2(N):
    x = sort(random.uniform(0, 2*pi, N))
    t = random.normal(sin(x),.2)
    return x,t

In [ ]:
x, t = gen_sinusoidal2(500)
plot(x, t, '+')

### 2.2 Compute Posterior (15 points)

You're going to implement a Bayesian linear regression model, and fit it to the sinusoidal data. Your regression model has a zero-mean isotropic Gaussian prior over the parameters, governed by a single (scalar) precision parameter $\alpha$, i.e.:

$$p(\bw \;|\; \alpha) = \mathcal{N}(\bw \;|\; 0, \alpha^{-1} \bI)$$

The covariance and mean of the posterior are given by:

$$\bS_N= \left( \alpha \bI + \beta \bPhi^T \bPhi \right)^{-1} $$
$$\bm_N = \beta\; \bS_N \bPhi^T \bt$$

where $\alpha$ is the precision of the predictive distribution, and $\beta$ is the noise precision. 
See MLPR chapter 3.3 for background.

Write a method `fit_polynomial_bayes(x, t, M, alpha, beta)` that returns the mean $\bm_N$ and covariance $\bS_N$ of the posterior for a $M$-th order polynomial, given a dataset, where `x`, `t` and `M` have the same meaning as in question 1.2.

In [ ]:
def fit_polynomial_bayes(x, t, M, alpha, beta):
    Phi = fliplr(vander(x, M+1))
    S_N = linalg.inv(alpha * eye(M+1) + beta * Phi.T.dot(Phi))
    m_N = beta * S_N.dot( Phi.T.dot(t) )
    return S_N, m_N

### 2.3 Prediction (10 points)

The predictive distribution of Bayesian linear regression is:

$$ p(t \;|\; \bx, \bt, \alpha, \beta) = \mathcal{N}(t \;|\; \bm_N^T \phi(\bx), \sigma_N^2(\bx))$$

$$ \sigma_N^2 = \frac{1}{\beta} + \phi(\bx)^T \bS_N \phi(\bx) $$

where $\phi(\bx)$ are the computed features for a new datapoint $\bx$, and $t$ is the predicted variable for datapoint $\bx$. 

Write a function that `predict_polynomial_bayes(x, m, S, beta)` that returns the predictive mean and variance given a new datapoint `x`, posterior mean `m`, posterior variance `S` and a choice of model variance `beta`.

In [ ]:
def predict_polynomial_bayes(x, m, S, beta):
    N = S.shape[0] # Note: N = M+1
    phi_x = fliplr(vander([x], N))[0]
    sigma = phi_x.T.dot(S).dot(phi_x)
    mu = m.T.dot(phi_x)
    return mu, sigma

### 2.4 Plot predictive distribution (10 points)

a) (5 points) Generate 7 datapoints with `gen_sinusoidal2(7)`. Compute the posterior mean and covariance for a Bayesian polynomial regression model with $M=5$, $\alpha=\frac{1}{2}$ and $\beta=\frac{1}{0.2^2}$.
Plot the Bayesian predictive distribution, where you plot (for $x$ between 0 and $2 \pi$) $t$'s predictive mean and a 1-sigma predictive variance using `plt.fill_between(..., alpha=0.1)` (the alpha argument induces transparency).

Include the datapoints in your plot.

b) (5 points) For a second plot, draw 100 samples from the parameters' posterior distribution. Each of these samples is a certain choice of parameters for 5-th order polynomial regression. 
Display each of these 100 polynomials.

In [ ]:
beta = 25.0
x, t = gen_sinusoidal2(7)
S_N, m_N = fit_polynomial_bayes(x, t, 5, .5, beta)

xs = linspace(0, 2*pi, 100)
# Matrix with mu's and sigma's as columns
stats = np.array([predict_polynomial_bayes(d, m_N, S_N, beta) for d in xs])

figsize(10,5)
plot(x, t, 'o')
plot(xs, sin(xs), '--', color='black')
plot(xs, stats[:,0], 'r-')
ylim(-1.5,1.5)
xlim(0,2*pi)
fill_between(xs, stats[:,0] + stats[:,1], stats[:,0] - stats[:,1], alpha=.2, color='r')

In [ ]:
def show_regression(x, t, w, xs=False):
    if xs == False:
        xs = linspace(0, 2*pi, 100)
    plot(xs, polyval(w[::-1], xs), '-r', lw=2)
    plot(x, t, 'o')
    plot(xs, sin(xs), '-g')
    show()

In [ ]:
beta = 25.0
x, t = gen_sinusoidal2(7)
S_N, m_N = fit_polynomial_bayes(x, t, 5, .5, beta)

xs = linspace(0, 2*pi, 100)
ws = random.multivariate_normal(m_N, S_N, 100)
for w in ws :
    plot(xs, polyval(w[::-1], xs), 'r-', alpha=.1)

figsize(10,5)
ylim(-2,2)
xlim(0,2*pi)
plot(x, t, 'o', color='black')
plot(xs, sin(xs), '-', color='black')
show()

### 2.5 Additional questions (10 points)

a) (5 points) Why is $\beta=\frac{1}{0.2^2}$ the best choice of $\beta$ in section 2.4?

b) (5 points) In the case of Bayesian linear regression, both the posterior of the parameters $p(\bw \;|\; \bt, \alpha, \beta)$ and the predictive distribution $p(t \;|\; \bw, \beta)$ are Gaussian. In consequence (and conveniently), $p(t \;|\; \bt, \alpha, \beta)$ is also Gaussian (See MLPR section 3.3.2 and homework 2 question 4). This is actually one of the (rare) cases where we can make Bayesian predictions without resorting to approximative methods.

Suppose you have to work with some model $p(t\;|\;x,\bw)$ with parameters $\bw$, where the posterior distribution $p(\bw\;|\;\mathcal{D})$ given dataset $\mathcal{D}$ can not be integrated out when making predictions, but where you can still generate samples from the posterior distribution of the parameters. Explain how you can still make approximate Bayesian predictions using samples from the parameters' posterior distribution.

## Answers
a) In section 2.4, we considered data points generated by the `gen_sinusoidal2` method. In its implementation, we considered a regression function $h(x) = \sin(x)$ (or $h(x) = \sin(2 \pi x)$, depending on whether you follow Bishop or the lab exercise) with target value deviation $\sigma = 0.2$. In the book, it is said that the precision $\beta$ should be set to $\beta = 1/\sigma^2$ (see page 24). This is supported by the deduction in page 142, equation (3.21): we see that maximizing the log likelihood of (3.11) wrt $\beta$ yields a $\beta_{ML}$ that is of exactly this form. So in conclusion, setting $\beta = 1/(0.2)^2$ is consistent with the theory.

b) We can use a sampling scheme (for example Markov chain Monte Carlo, chapter 11.2) to draw a set $z^{(l)}$ of $L$ independent samples from $p(\bw\;|\;\mathcal{D})$. Now we can approximate $p(t \;|\; \bt, \alpha, \beta)$ with $\frac{1}{L}\sum_{l=1}^L p(t \;|\; z^{(l)}, \beta)$. This is analogous to equation (11.2). Therefore, this approximation has the correct expected value.